In [1]:
from bs4 import BeautifulSoup
import urllib.request
import re
import requests
import numpy as np
import spacy
import time
nlp = spacy.load('en')

In [2]:
def match_num(text):
    number_words = ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'eleven', 'twelve', 'thirteen', 'fourteen', 'fifteen', 'sixteen', 'seventeen', 'eighteen', 'nineteen', 'twenty', 'thirty', 'forty', 'fifty', 'sixty', 'seventy', 'eighty', 'ninety', 'hundred','hundreds', 'thousand','thousands']
    has_digit = re.search('\d+',text)
    if not has_digit:
        for word in number_words:
            pattern = '\b' + word + '\b'
            if re.search('pattern',text):
                has_digit = True
                break
    return has_digit
    

In [3]:
resp = urllib.request.urlopen("https://globalnews.ca/?s=alberta+coronavirus")
soup = BeautifulSoup(resp, from_encoding=resp.info().get_param('charset'))
link_list = []
pattern = 'https://globalnews.ca/news/\d+/.*/$'
for link in soup.find_all('a', href=True):
    if re.match(pattern,link['href']):
        link_list.append(link['href'])
    link_set = set(link_list)
unique_link_list = list(link_set)

In [5]:
for link in unique_link_list:
    time.sleep(1)
    article = requests.get(link)
    article_content = article.content
    soup_article = BeautifulSoup(article_content, 'html.parser')
    # Show date
    post_time =soup_article.find_all('div', class_='c-byline__date')
    date =[]
    for t in post_time:
        date.append(t.get_text())
    # Show number related phrase
    body = soup_article.find_all('div')#, class_='l-article__part'
    x = body[0].find_all('p')
    news_contents = ''
    # Unifying the paragraphs
    list_paragraphs = []
    for p in np.arange(0, len(x)):
        paragraph = x[p].get_text()
        if not 'read more' in  paragraph.lower():
            list_paragraphs.append(paragraph)
        final_article = " ".join(list_paragraphs)
    spacy_doc = nlp(final_article)
    related_text = ""
    for sent in spacy_doc.sents:
        if 'case' in sent.text and 'new' in sent.text:
            if match_num(sent.text):
                clean_text = sent.text.replace('\n',' ')
                clean_text = clean_text.replace('\t','')
                related_text =related_text + clean_text + '\n'
    if related_text:
        print(soup_article.title.get_text())
        print(date)
        print(related_text)

74 confirmed coronavirus cases in Alberta, cases detected in all health zones  | Globalnews.ca
['Posted March 16, 2020 6:57 pm', 'Updated March 17, 2020 11:32 am']
Dr. Deena Hinshaw said there were 18 new cases of COVID-19, bringing the total number to 74 across the province.

Premier Jason Kenney declares COVID-19 public health emergency in Alberta  | Globalnews.ca
['Posted March 17, 2020 3:04 pm', 'Updated March 17, 2020 11:40 pm']
At a news conference on Tuesday afternoon, Alberta’s chief medical officer of health said the province has 23 new confirmed cases of COVID-19, bringing the total up to 97 and up from 74 a day earlier.

Alberta’s 2nd presumptive case of coronavirus is in Edmonton; Calgary closes 2 ATB branches  | Globalnews.ca
['Posted March 6, 2020 7:43 pm', 'Updated March 7, 2020 7:26 pm']
Alberta’s chief medical health officer, Dr. Deena Hinshaw, said at a news conference Friday that the second case is connected to a man in his 40s from Edmonton who just returned from a 